# Notebook to Compare 2 R-Maps using Permutation
- Notes on controlling a regression:
    - Adding covariates to a regression will 'control' for them, but will almost always increase the R-squared. 
    - To 'remove' a covariate from the regression, you will want to regress a nuisance covariate OUT of the covariate of interest. 
        - This means your regressor will become the residuals from the regression of cov_1 ~ nuisance_cov1

# Get Dataset One

Import Niftis
- These are EXPECTED to have subject IDs which are IDENTICAL to the subject IDs that go in the covarite DF column names below
- Column labels are subject IDs. 
- This is expected to ultimately have the form:

|        |  1 |  2 |  3 |  4 |  5 |  6 |  7 |  8 |  9 |  10 | ... |  40 |  41 |  42 |  43 |  45 |  46 |  47 |  48 |  49 |  50 |
|----------|------------|-----------|-----------|-----------|-----------|-----------|-----------|-----------|-----------|------------|-----|-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|
| Voxel 1     | 3          | 4         | 7         | 2         | 2         | 2         | 9         | 4         | 7         | 5          | ... | 5           | 2           | 7           | 7           | 3           | 8           | 8           | 1           | 1           | 3           |
| . . .      | ...         | ...        | ...         | ...         | ...         | ...         | ...         | ...         | ...         | ...          | ... | ...           | ...           | ...           | ...           | ...           | ...           | 7           | ...           | ...           | ...           |
| Voxel N     | 2          | 1         | 0         | 1         | 3         | 4         | 9         | 5         | 8         | 6          | ... | 6           | 3           | 8           | 8           | 4           | 9           | 9           | 2           | 2           | 4           |

In [140]:
import_path = '/Volumes/Expansion/datasets/adni/neuroimaging/true_ad_randomized/connectivity/seeded_connectivity'
file_target = '*_T.nii*'

In [144]:
from calvin_utils.file_utils.import_functions import GiiNiiFileImport
giinii = GiiNiiFileImport(import_path=import_path, file_column=None, file_pattern=file_target)
nimg_df = giinii.run()
nimg_df

Attempting to import from: /Volumes/Expansion/datasets/adni/neuroimaging/true_ad_randomized/connectivity/seeded_connectivity/*_T.nii*


,sub-subh0001ucomposite_space-2mm_lesionMask_T.nii.gz,sub-subh0002ucomposite_space-2mm_lesionMask_T.nii.gz,sub-subh0003ucomposite_space-2mm_lesionMask_T.nii.gz,sub-subh0004ucomposite_space-2mm_lesionMask_T.nii.gz,sub-subh0005ucomposite_space-2mm_lesionMask_T.nii.gz,sub-subh0006ucomposite_space-2mm_lesionMask_T.nii.gz,sub-subh0007ucomposite_space-2mm_lesionMask_T.nii.gz,sub-subh0008ucomposite_space-2mm_lesionMask_T.nii.gz,sub-subh0009ucomposite_space-2mm_lesionMask_T.nii.gz,sub-subh0010ucomposite_space-2mm_lesionMask_T.nii.gz,...,sub-subh0041ucomposite_space-2mm_lesionMask_T.nii.gz,sub-subh0042ucomposite_space-2mm_lesionMask_T.nii.gz,sub-subh0043ucomposite_space-2mm_lesionMask_T.nii.gz,sub-subh0044ucomposite_space-2mm_lesionMask_T.nii.gz,sub-subh0045ucomposite_space-2mm_lesionMask_T.nii.gz,sub-subh0046ucomposite_space-2mm_lesionMask_T.nii.gz,sub-subh0047ucomposite_space-2mm_lesionMask_T.nii.gz,sub-subh0048ucomposite_space-2mm_lesionMask_T.nii.gz,sub-subh0049ucomposite_space-2mm_lesionMask_T.nii.gz,sub-subh0050ucomposite_space-2mm_lesionMask_T.nii.gz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902624,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902626,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902627,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Fix names

In [145]:
pre = 'sub-subh'
post = 'ucomposite'

In [146]:
nimg_df = GiiNiiFileImport.splice_colnames(nimg_df, pre, post)
nimg_df

,0001,0002,0003,0004,0005,0006,0007,0008,0009,0010,...,0041,0042,0043,0044,0045,0046,0047,0048,0049,0050
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902624,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902626,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902627,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Import Covariates

**The CSV is expected to be in this format**
- sub column contents MUST match the names of the neuroimaging files above. 
    - ID column 
```
+-----+----------------------------+--------------+--------------+--------------+
| sub | Nifti_File_Path            | Covariate_1  | Covariate_2  | Covariate_3  |
+-----+----------------------------+--------------+--------------+--------------+
| 1   | /path/to/file1.nii.gz      | 0.5          | 1.2          | 3.4          |
| 2   | /path/to/file2.nii.gz      | 0.7          | 1.4          | 3.1          |
| 3   | /path/to/file3.nii.gz      | 0.6          | 1.5          | 3.5          |
| 4   | /path/to/file4.nii.gz      | 0.9          | 1.1          | 3.2          |
| ... | ...                        | ...          | ...          | ...          |
+-----+----------------------------+--------------+--------------+--------------+
```

In [152]:
# Specify the path to your CSV file containing NIFTI paths
input_csv_path = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/atrophy_seeds_2023/metadata/true_ad_patient_data/z0_atrophy_thresholded.csv'
sheet= None #'FCS_Demographics_and_Behavior.c' #'Memory'

In [153]:
# Specify where you want to save your results to
out_dir = '/Volumes/Expansion/datasets/Manitoba_PET/PET_R_Maps/autonocog_memory/similarity_to_lesion_memory_net'

In [154]:
from calvin_utils.permutation_analysis_utils.statsmodels_palm import CalvinStatsmodelsPalm
# Instantiate the PalmPrepararation class
cal_palm = CalvinStatsmodelsPalm(input_csv_path=input_csv_path, output_dir=out_dir, sheet=sheet)
# Call the process_nifti_paths method
data_df = cal_palm.read_and_display_data()

,subject,Z0_CSF_Cerebellum,Z0_CSF_Cortex,Z0_CSF_Subcortex,Z0_CSF_Temporal,Z0_CSF_Brain,Z0_CSF_MTL,Z0_CSF_Occipital,Z0_CSF_Frontal,Z0_CSF_Parietal,...,Q7,Q8,Q9,Q10,Q11,Q12,Q14,TOTAL11,TOTALMOD,Diagnosis
0,1,23250.988140,71399.71890,4579.425184,20701.60782,103758.33050,193.448280,12271.74449,25644.86940,19903.50654,...,2,4,0,0,0,0,0,9.67,18.67,Alzheimer
1,2,22958.897280,138347.01900,11127.637900,36840.85875,183216.75810,948.495659,36745.48487,36589.92827,38646.69415,...,0,7,0,0,0,0,4,16.67,29.67,Alzheimer
2,3,26161.834440,130676.44860,5533.180865,28298.62220,165779.02460,437.670737,35581.06790,34791.90545,40958.99248,...,0,12,1,0,0,0,2,18.33,29.33,Alzheimer
3,4,31189.302650,195604.92070,33125.168810,48863.28672,264363.91590,1072.287735,43774.45307,60989.63249,60430.17254,...,2,5,0,0,0,0,2,11.67,20.67,Alzheimer
4,5,36518.900570,158015.21760,9475.521590,41949.05563,206790.29380,1356.513850,19351.39418,69530.68767,36202.65540,...,2,12,5,0,0,0,5,32.33,45.33,Alzheimer
5,6,59637.380890,243616.16270,25565.841520,66725.04776,341740.12280,1960.569291,47480.69252,76924.48785,71056.80037,...,3,2,0,0,0,0,3,15.00,26.00,Alzheimer
6,7,9621.489330,54820.27148,5525.381934,12683.80400,71067.33222,182.958270,15748.10757,13408.35527,16792.10843,...,2,5,0,1,1,1,2,17.67,29.67,Alzheimer
7,8,15261.407690,85382.76423,4719.331915,23189.77273,115110.09680,654.167430,16992.27795,30348.99846,22405.64365,...,2,5,0,0,0,0,0,13.67,23.67,Alzheimer
8,9,13111.582450,110324.62620,6013.140341,36133.85243,136350.71310,614.484539,18163.27931,35974.74061,27887.03209,...,3,2,1,0,0,0,4,14.33,27.33,Alzheimer
9,10,21814.021110,195050.57000,16494.508520,53673.24458,248034.55240,1657.605565,40384.14119,64786.91266,52184.19098,...,5,11,0,0,0,0,2,23.67,35.67,Alzheimer


In [155]:
variable_of_interest = 'Q4'

In [156]:
# data_df['subject'] = data_df['subject'].str[4:]
data_df

,subject,Z0_CSF_Cerebellum,Z0_CSF_Cortex,Z0_CSF_Subcortex,Z0_CSF_Temporal,Z0_CSF_Brain,Z0_CSF_MTL,Z0_CSF_Occipital,Z0_CSF_Frontal,Z0_CSF_Parietal,...,Q7,Q8,Q9,Q10,Q11,Q12,Q14,TOTAL11,TOTALMOD,Diagnosis
0,1,23250.988140,71399.71890,4579.425184,20701.60782,103758.33050,193.448280,12271.74449,25644.86940,19903.50654,...,2,4,0,0,0,0,0,9.67,18.67,Alzheimer
1,2,22958.897280,138347.01900,11127.637900,36840.85875,183216.75810,948.495659,36745.48487,36589.92827,38646.69415,...,0,7,0,0,0,0,4,16.67,29.67,Alzheimer
2,3,26161.834440,130676.44860,5533.180865,28298.62220,165779.02460,437.670737,35581.06790,34791.90545,40958.99248,...,0,12,1,0,0,0,2,18.33,29.33,Alzheimer
3,4,31189.302650,195604.92070,33125.168810,48863.28672,264363.91590,1072.287735,43774.45307,60989.63249,60430.17254,...,2,5,0,0,0,0,2,11.67,20.67,Alzheimer
4,5,36518.900570,158015.21760,9475.521590,41949.05563,206790.29380,1356.513850,19351.39418,69530.68767,36202.65540,...,2,12,5,0,0,0,5,32.33,45.33,Alzheimer
5,6,59637.380890,243616.16270,25565.841520,66725.04776,341740.12280,1960.569291,47480.69252,76924.48785,71056.80037,...,3,2,0,0,0,0,3,15.00,26.00,Alzheimer
6,7,9621.489330,54820.27148,5525.381934,12683.80400,71067.33222,182.958270,15748.10757,13408.35527,16792.10843,...,2,5,0,1,1,1,2,17.67,29.67,Alzheimer
7,8,15261.407690,85382.76423,4719.331915,23189.77273,115110.09680,654.167430,16992.27795,30348.99846,22405.64365,...,2,5,0,0,0,0,0,13.67,23.67,Alzheimer
8,9,13111.582450,110324.62620,6013.140341,36133.85243,136350.71310,614.484539,18163.27931,35974.74061,27887.03209,...,3,2,1,0,0,0,4,14.33,27.33,Alzheimer
9,10,21814.021110,195050.57000,16494.508520,53673.24458,248034.55240,1657.605565,40384.14119,64786.91266,52184.19098,...,5,11,0,0,0,0,2,23.67,35.67,Alzheimer


**Preprocess Your Data**

**Handle NANs**
- Set drop_nans=True is you would like to remove NaNs from data
- Provide a column name or a list of column names to remove NaNs from

In [157]:
# data_df.columns

In [158]:
drop_list = [variable_of_interest]

In [159]:
data_df = cal_palm.drop_nans_from_columns(columns_to_drop_from=drop_list)
display(data_df)

,subject,Z0_CSF_Cerebellum,Z0_CSF_Cortex,Z0_CSF_Subcortex,Z0_CSF_Temporal,Z0_CSF_Brain,Z0_CSF_MTL,Z0_CSF_Occipital,Z0_CSF_Frontal,Z0_CSF_Parietal,...,Q7,Q8,Q9,Q10,Q11,Q12,Q14,TOTAL11,TOTALMOD,Diagnosis
0,1,23250.988140,71399.71890,4579.425184,20701.60782,103758.33050,193.448280,12271.74449,25644.86940,19903.50654,...,2,4,0,0,0,0,0,9.67,18.67,Alzheimer
1,2,22958.897280,138347.01900,11127.637900,36840.85875,183216.75810,948.495659,36745.48487,36589.92827,38646.69415,...,0,7,0,0,0,0,4,16.67,29.67,Alzheimer
2,3,26161.834440,130676.44860,5533.180865,28298.62220,165779.02460,437.670737,35581.06790,34791.90545,40958.99248,...,0,12,1,0,0,0,2,18.33,29.33,Alzheimer
3,4,31189.302650,195604.92070,33125.168810,48863.28672,264363.91590,1072.287735,43774.45307,60989.63249,60430.17254,...,2,5,0,0,0,0,2,11.67,20.67,Alzheimer
4,5,36518.900570,158015.21760,9475.521590,41949.05563,206790.29380,1356.513850,19351.39418,69530.68767,36202.65540,...,2,12,5,0,0,0,5,32.33,45.33,Alzheimer
5,6,59637.380890,243616.16270,25565.841520,66725.04776,341740.12280,1960.569291,47480.69252,76924.48785,71056.80037,...,3,2,0,0,0,0,3,15.00,26.00,Alzheimer
6,7,9621.489330,54820.27148,5525.381934,12683.80400,71067.33222,182.958270,15748.10757,13408.35527,16792.10843,...,2,5,0,1,1,1,2,17.67,29.67,Alzheimer
7,8,15261.407690,85382.76423,4719.331915,23189.77273,115110.09680,654.167430,16992.27795,30348.99846,22405.64365,...,2,5,0,0,0,0,0,13.67,23.67,Alzheimer
8,9,13111.582450,110324.62620,6013.140341,36133.85243,136350.71310,614.484539,18163.27931,35974.74061,27887.03209,...,3,2,1,0,0,0,4,14.33,27.33,Alzheimer
9,10,21814.021110,195050.57000,16494.508520,53673.24458,248034.55240,1657.605565,40384.14119,64786.91266,52184.19098,...,5,11,0,0,0,0,2,23.67,35.67,Alzheimer


**Drop Row Based on Value of Column**

Define the column, condition, and value for dropping rows
- column = 'your_column_name'
- condition = 'above'  # Options: 'equal', 'above', 'below'

Set the parameters for dropping rows

In [160]:
# column = 'redcap_event_name'  # The column you'd like to evaluate
# condition = 'not'  # Thecondition to check ('equal', 'above', 'below', 'not')
# value = '1year_arm_1' # The value to compare against

In [161]:
# data_df, other_df = cal_palm.drop_rows_based_on_value(column, condition, value)
# data_df

Regress out a Covariate

In [162]:
# lis = []
# for col in data_df.columns:
#     if 'surface' in col.lower():
#         lis.append(col)
# print(lis)

In [163]:
# from calvin_utils.statistical_utils.regression_utils import RegressOutCovariates
## use this code block to regress out covariates. Generally better to just include as covariates in a model..
# dependent_variable_list = lis
# regressors = ['Age', 'Sex']

# data_df, adjusted_dep_vars_list = RegressOutCovariates.run(df=data_df, dependent_variable_list=dependent_variable_list, covariates_list=regressors)
# print(adjusted_dep_vars_list)

**Standardize Data**
- Enter Columns you Don't want to standardize into a list

In [164]:
## Remove anything you don't want to standardize
# cols_not_to_standardize = ['Age',  'Subiculum_Connectivity_T']

In [165]:
# data_df = cal_palm.standardize_columns(cols_not_to_standardize)
# data_df

Choose Rows to Keep
- Keep your subject row and your dependent variable

In [166]:
col_to_keep_list = [variable_of_interest, 'subject']

- The final DF is EXPECTED to have subject IDs which are IDENTICAL to the subject IDs that go in the neuroimaging DF column names above
- There should only be 1 variable  the row

|        |  1 |  2 |  3 |  4 |  5 |  6 |  7 |  8 |  9 |  10 | ... |  40 |  41 |  42 |  43 |  45 |  46 |  47 |  48 |  49 |  50 |
|----------|------------|-----------|-----------|-----------|-----------|-----------|-----------|-----------|-----------|------------|-----|-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|
| Indep. Var.    | 3          | 4         | 7         | 2         | 2         | 2         | 9         | 4         | 7         | 5          | ... | 5           | 2           | 7           | 7           | 3           | 8           | 8           | 1           | 1           | 3           |

In [167]:
data_df=data_df.loc[:, col_to_keep_list]
data_df = data_df.T
data_df.columns = data_df.loc['subject', :]
data_df = data_df.drop('subject')
data_df.dropna(inplace=True, axis=1)
data_df

subject,1,2,3,4,5,6,7,8,9,10,...,41,42,43,44,45,46,47,48,49,50
Q4,9,9,9,7,8,8,10,10,9,10,...,8,9,10,10,7,10,10,9,10,10


# Get Dataset Two

Import Niftis
- These are EXPECTED to have subject IDs which are IDENTICAL to the subject IDs that go in the covarite DF column names below
- Column labels are subject IDs. 
- This is expected to ultimately have the form:

|        |  1 |  2 |  3 |  4 |  5 |  6 |  7 |  8 |  9 |  10 | ... |  40 |  41 |  42 |  43 |  45 |  46 |  47 |  48 |  49 |  50 |
|----------|------------|-----------|-----------|-----------|-----------|-----------|-----------|-----------|-----------|------------|-----|-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|
| Voxel 1     | 3          | 4         | 7         | 2         | 2         | 2         | 9         | 4         | 7         | 5          | ... | 5           | 2           | 7           | 7           | 3           | 8           | 8           | 1           | 1           | 3           |
| . . .      | ...         | ...        | ...         | ...         | ...         | ...         | ...         | ...         | ...         | ...          | ... | ...           | ...           | ...           | ...           | ...           | ...           | 7           | ...           | ...           | ...           |
| Voxel N     | 2          | 1         | 0         | 1         | 3         | 4         | 9         | 5         | 8         | 6          | ... | 6           | 3           | 8           | 8           | 4           | 9           | 9           | 2           | 2           | 4           |

In [168]:
# import_path2 = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/BIDS_AD_DBS_FORNIX/connectivity_data/vta_published_t_connectivity'
# file_target2 = '*T*'

In [169]:
# from calvin_utils.file_utils.import_functions import GiiNiiFileImport
# giinii2 = GiiNiiFileImport(import_path=import_path2, file_column=None, file_pattern=file_target2)
# nimg_df2 = giinii2.run()
# nimg_df2

Fix names

In [170]:
# pre2 = ''
# post2 = '_vat'

In [171]:
# nimg_df2 = GiiNiiFileImport.splice_colnames(nimg_df2, pre2, post2)
# nimg_df2

Import Covariates

**The CSV is expected to be in this format**
- sub column contents MUST match the names of the neuroimaging files above. 
    - ID column 
```
+-----+----------------------------+--------------+--------------+--------------+
| sub | Nifti_File_Path            | Covariate_1  | Covariate_2  | Covariate_3  |
+-----+----------------------------+--------------+--------------+--------------+
| 1   | /path/to/file1.nii.gz      | 0.5          | 1.2          | 3.4          |
| 2   | /path/to/file2.nii.gz      | 0.7          | 1.4          | 3.1          |
| 3   | /path/to/file3.nii.gz      | 0.6          | 1.5          | 3.5          |
| 4   | /path/to/file4.nii.gz      | 0.9          | 1.1          | 3.2          |
| ... | ...                        | ...          | ...          | ...          |
+-----+----------------------------+--------------+--------------+--------------+
```

In [172]:
# # Specify the path to your CSV file containing NIFTI paths
# input_csv_path2 = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/cognition_2023/metadata/master_list_proper_subjects.xlsx'
# sheet2 = 'master_list_proper_subjects'

In [173]:
# from calvin_utils.permutation_analysis_utils.statsmodels_palm import CalvinStatsmodelsPalm
# # Instantiate the PalmPrepararation class
# cal_palm2 = CalvinStatsmodelsPalm(input_csv_path=input_csv_path2, output_dir=out_dir, sheet=sheet2)
# # Call the process_nifti_paths method
# data_df2 = cal_palm2.read_and_display_data()

**Preprocess Your Data**

**Handle NANs**
- Set drop_nans=True is you would like to remove NaNs from data
- Provide a column name or a list of column names to remove NaNs from

In [174]:
# data_df2.columns

In [175]:
# drop_list2 = ['Z_Scored_Percent_Cognitive_Improvement']

In [176]:
# data_df2 = cal_palm2.drop_nans_from_columns(columns_to_drop_from=drop_list2)
# display(data_df2)

**Drop Row Based on Value of Column**

Define the column, condition, and value for dropping rows
- column = 'your_column_name'
- condition = 'above'  # Options: 'equal', 'above', 'below'

Set the parameters for dropping rows

In [177]:
# column2 = 'City'  # The column you'd like to evaluate
# condition2 = 'not'  # Thecondition to check ('equal', 'above', 'below', 'not')
# value2 = 'Toronto' # The value to compare against

In [178]:
# data_df2, other_df2 = cal_palm2.drop_rows_based_on_value(column2, condition2, value2)
# data_df2

Regress out a Covariate

In [179]:
# lis = []
# for col in data_df2.columns:
#     if 'surface' in col.lower():
#         lis.append(col)
# print(lis)

In [180]:
from calvin_utils.statistical_utils.regression_utils import RegressOutCovariates
## use this code block to regress out covariates. Generally better to just include as covariates in a model..
# dependent_variable_list2 = lis
# regressors2 = ['Age', 'Sex']

# data_df2, adjusted_dep_vars_list2 = RegressOutCovariates.run(df=data_df2, dependent_variable_list=dependent_variable_list2, covariates_list=regressors2)
# print(adjusted_dep_vars_list2)

**Standardize Data**
- Enter Columns you Don't want to standardize into a list

In [181]:
## Remove anything you don't want to standardize
# cols_not_to_standardize2 = ['Age',  'Subiculum_Connectivity_T']

In [182]:
# data_df2 = cal_palm2.standardize_columns(cols_not_to_standardize2)
# data_df2

Choose Rows to Keep
- Keep subject and dependent variable row

In [183]:
# col_to_keep_list2 = ['Z_Scored_Percent_Cognitive_Improvement', 'subject']

- The final DF is EXPECTED to have subject IDs which are IDENTICAL to the subject IDs that go in the neuroimaging DF column names above
- There should only be 1 variable  the row

|        |  1 |  2 |  3 |  4 |  5 |  6 |  7 |  8 |  9 |  10 | ... |  40 |  41 |  42 |  43 |  45 |  46 |  47 |  48 |  49 |  50 |
|----------|------------|-----------|-----------|-----------|-----------|-----------|-----------|-----------|-----------|------------|-----|-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|
| Indep. Var.    | 3          | 4         | 7         | 2         | 2         | 2         | 9         | 4         | 7         | 5          | ... | 5           | 2           | 7           | 7           | 3           | 8           | 8           | 1           | 1           | 3           |

In [184]:
# data_df2=data_df2.loc[:, col_to_keep_list2]
# data_df2 = data_df2.T
# data_df2.columns = data_df2.loc['subject']
# data_df2 = data_df2.drop('subject')
# data_df2.dropna(inplace=True, axis=1)
# data_df2

# Define an Already Existing Map to Compare Similarity To
- if not using, set to None

In [185]:
map_path = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/published_networks/niftis/Memory Network T Conn.nii'

data_df2=None
nimg_df2=None

# Test 2 Maps

# Prepare the Arguments for Permutation Testing

Is there a particular mask you want to use?
- MUST match the resolution of voxelwise data being analyzed. 
- If you set None, the voxelwise data will be used for thresholding. 
    - Values below mask_threshold (float) will be set to 0. 
- Warning: bad masking may result in failed experiments. Erroneous voxels outside the brain will influence the correction. 

In [186]:
mask_path = '/Users/cu135/hires_backdrops/MNI152_T1_2mm_brain_mask.nii'
mask_threshold = 0

Correlation method
- spearman or pearson

In [187]:
method = 'spearman'

Choose Max Stat Correction Method
- None | pseudo_var_smooth | var_smooth

In [188]:
max_stat_method = 'pseudo_var_smooth'

ROI to analyze within

In [189]:
roi_path = None
roi_threshold = 0

Initialize the Permutation testing Class

In [190]:
from calvin_utils.permutation_analysis_utils.correlation_fwe_comparison import CalvinFWEWrapper
wrapper = CalvinFWEWrapper(neuroimaging_dataframe1=nimg_df, 
                           variable_dataframe1=data_df, 
                           neuroimaging_dataframe2=nimg_df2, 
                           variable_dataframe2=data_df2, 
                           mask_threshold=mask_threshold, 
                           mask_path=mask_path, 
                           out_dir=out_dir, 
                           method=method, 
                           max_stat_method=max_stat_method,
                           roi_path=roi_path, roi_threshold=0,
                           map_path=map_path, use_spearman=True,
                           two_tail=False)

Analyze the Similarity of the 2 maps

In [192]:
# Running Pearson correlation analysis with ROI mask
observed_correlation, permuted_correlations = wrapper.run_pearson_analysis(n_permutations=1000)

Running permutation: 100%|██████████| 1000/1000 [08:24<00:00,  1.98it/s]

Observed: 0.3127601421809797, p-value 0.843, using 2-tail: False.


Analyze the Peaks of the Two Maps

In [139]:
# Running peak voxel finding analysis with ROI mask
observed_peak_distance, permuted_peak_distances = wrapper.run_peak_voxel_analysis(n_permutations=1000)

AttributeError: 'CalvinFWEWrapper' object has no attribute 'roi_data_3d'

In [ ]:
from scipy.stats import percentileofscore
test_value = 2
percentile = percentileofscore(permuted_peak_distances, test_value, kind='rank')
percentile